<a href="https://colab.research.google.com/github/DB9653/Bachelorarbeit/blob/main/Cross_Topic_Modeling_GE18_allgemein.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTS


*   pandas
*   torch
*   numpy
*   bertopic
*   ~~spacy~~
*   string
*   regex
*   sidetable
*   nltk





In [ ]:
import pandas as pd

In [ ]:
pip show pandas

Name: pandas
Version: 1.3.5
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.7/dist-packages
Requires: numpy, pytz, python-dateutil
Required-by: xarray, vega-datasets, statsmodels, sklearn-pandas, sidetable, seaborn, pymc, prophet, plotnine, pandas-profiling, pandas-gbq, pandas-datareader, mlxtend, mizani, holoviews, gspread-dataframe, google-colab, fix-yahoo-finance, fastai, datascience, cufflinks, cmdstanpy, bertopic, arviz, altair


In [ ]:
import torch

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [ ]:
device

device(type='cuda')

In [ ]:
!pip install bertopic
from bertopic import BERTopic
#model_for_whole_dataset = BERTopic(language="german", nr_topics="auto")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np

In [ ]:
import string
import regex as re
!pip install sidetable
import sidetable

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Stopwords
import nltk 
nltk.download('omw-1.4')
#nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stopwords=set(stopwords.words('german'))



[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
add_stopwords = {'lbr', 'amp', 'schon', 'mal', 'mehr', 'einfach'} 
stopwords.update(add_stopwords)
stopwords

{'aber',
 'alle',
 'allem',
 'allen',
 'aller',
 'alles',
 'als',
 'also',
 'am',
 'amp',
 'an',
 'ander',
 'andere',
 'anderem',
 'anderen',
 'anderer',
 'anderes',
 'anderm',
 'andern',
 'anderr',
 'anders',
 'auch',
 'auf',
 'aus',
 'bei',
 'bin',
 'bis',
 'bist',
 'da',
 'damit',
 'dann',
 'das',
 'dass',
 'dasselbe',
 'dazu',
 'daß',
 'dein',
 'deine',
 'deinem',
 'deinen',
 'deiner',
 'deines',
 'dem',
 'demselben',
 'den',
 'denn',
 'denselben',
 'der',
 'derer',
 'derselbe',
 'derselben',
 'des',
 'desselben',
 'dessen',
 'dich',
 'die',
 'dies',
 'diese',
 'dieselbe',
 'dieselben',
 'diesem',
 'diesen',
 'dieser',
 'dieses',
 'dir',
 'doch',
 'dort',
 'du',
 'durch',
 'ein',
 'eine',
 'einem',
 'einen',
 'einer',
 'eines',
 'einfach',
 'einig',
 'einige',
 'einigem',
 'einigen',
 'einiger',
 'einiges',
 'einmal',
 'er',
 'es',
 'etwas',
 'euch',
 'euer',
 'eure',
 'eurem',
 'euren',
 'eurer',
 'eures',
 'für',
 'gegen',
 'gewesen',
 'hab',
 'habe',
 'haben',
 'hat',
 'hatte',




---



# Datensätze laden

**Dieser Schritt liefert uns den Datensatz mit den Einträgen aller vier Benchmarks:**  
*   HASOC_AND_GERMEVAL

**Weiterhin liegt jeder Benchmarkdatensatz in einem eigenen DataFrame vor:**  
*   df_hasoc2019
*   df_hasoc2020
*   df_germeval2018
*   df_germeval2019






  
  



## HASOC

### HASOC2019 TRAIN

In [ ]:
hasoc19train = pd.read_csv('german_dataset2019.tsv', sep='\t', encoding='UTF-8')

In [ ]:
hasoc19train.shape

(3819, 4)

In [ ]:
hasoc19train


,text_id,text,task_1,task_2
0,hasoc_de_1,Frank Rennicke – Ich bin stolz https://t.co/Cm...,NOT,NONE
1,hasoc_de_2,ANSEHEN.....und danach bitte TEILEN...TEILEN.....,NOT,NONE
2,hasoc_de_3,#Koeln Mohamed erkennt kein deutsches Recht so...,NOT,NONE
3,hasoc_de_4,#SaudiArabien ist eine brutale islamische Dikt...,NOT,NONE
4,hasoc_de_5,Bundespolizei #München hat im 1. Quartal 2019 ...,NOT,NONE
...,...,...,...,...
3814,hasoc_de_3815,"akquirieren Männer,die sich um die Kinder kümm...",NOT,NONE
3815,hasoc_de_3816,"Ja,schon seit Jahren! Ich muß dran denken,das...",NOT,NONE
3816,hasoc_de_3817,@Kurkamp @wendt_joachim @Schroeder_Live @lamni...,NOT,NONE
3817,hasoc_de_3818,EU Wahlen! AfD für Kindererziehungszeiten für...,NOT,NONE


### HASOC2019 TEST

In [ ]:
hasoc19test = pd.read_csv('hasoc_de_test_gold.tsv', sep='\t', encoding='UTF-8')
hasoc19test

,text_id,text,task_1,task_2
0,hasoc_de_5270,"Die Grüne Intelligenz Bestie 'Eckhardt', Prang...",HOF,HATE
1,hasoc_de_5271,"Das Deutschlandlied, Sinn und Unsinn https:/...",NOT,NONE
2,hasoc_de_5272,#Eurovision https://t.co/24T8fzUI44,NOT,NONE
3,hasoc_de_5273,Unsere ESC Kandidatinnen haben selbst Schuld a...,HOF,OFFN
4,hasoc_de_5274,Scheinbar bekommt Sarah's Conner's Sohn Vincen...,HOF,OFFN
...,...,...,...,...
845,hasoc_de_6115,#Erdogan ruft seine Mops wegen Untauglichkeit ...,HOF,OFFN
846,hasoc_de_6116,Ich würde der Linken Chefin Kipping die 1.000 ...,HOF,OFFN
847,hasoc_de_6117,Ausländer sind Top &amp; Trumpf Schon aufgefa...,HOF,OFFN
848,hasoc_de_6118,Wie der Regierungsjet die Regierungsflieger wi...,HOF,OFFN


### HASOC2020 TRAIN

In [ ]:
hasoc20train = pd.read_excel('hasoc_2020_de_train_new_a.xlsx', converters={'tweet_id':int})
hasoc20train

,tweet_id,text,task1,task2,ID
0,1133388798925189122,Deutsche rothaarige porno reife deutsche fraue...,NOT,NONE,hasoc_2020_de_2684
1,1127134592517980161,RT @NDRinfo: Die deutsche Klimaaktivistin Luis...,NOT,NONE,hasoc_2020_de_1042
2,1128897106171842560,@ruhrbahn jeden Morgen eine neue „Fahrzeugstör...,NOT,NONE,hasoc_2020_de_774
3,1123576753199484928,@Junge_Freiheit Die Inkas hatten sich schon dä...,NOT,NONE,hasoc_2020_de_559
4,1128743783393312768,RT @technosteron: leute die 'boar' schreiben l...,HOF,PRFN,hasoc_2020_de_1969
...,...,...,...,...,...
2368,1129340066604699649,"in conclusion, ich bin dumm",NOT,NONE,hasoc_2020_de_1840
2369,1133032786397409280,RT @chrisjuko: Mit viel Geld an die Macht \n#S...,NOT,NONE,hasoc_2020_de_345
2370,1125397127251804162,@FritzAlter1 @MiriamOzen Ich habe den Eindruck...,NOT,NONE,hasoc_2020_de_700
2371,1128318292228222982,RT @LucaBBM: Wieso hab ich immer Stress mit so...,HOF,PRFN,hasoc_2020_de_2167


### HASOC2020 TEST

In [ ]:
hasoc20test = pd.read_csv('hasoc20test_improved.csv', sep=',', encoding='UTF-8')
hasoc20test

,Unnamed: 0,tweet_id,text,task1,task2,ID
0,0,1129095874242650112,@derCarsti Boykottieren hört sich besser an. 💙💙,NOT,NONE,hasoc_2020_de_1053
1,1,1129004308396236800,"RT @ibikus31: Es wird spekuliert, ob Merkel ei...",NOT,NONE,hasoc_2020_de_671
2,2,1130896929355907080,Hat #Hitler wirklich den Krieg in der Wüste ve...,NOT,NONE,hasoc_2020_de_2977
3,3,1132251534329307136,RT @Beatrix_vStorch: #May tritt in UK unter Tr...,NOT,NONE,hasoc_2020_de_1746
4,4,1124941869115498496,@justmeDoro Eher nicht. Das Gänse hauen wieder...,NOT,NONE,hasoc_2020_de_2416
...,...,...,...,...,...,...
521,521,1124809878546128897,RT @ChanMachtSo: SCHMERZEN!!!! Au!!! Mein Gehi...,NOT,NONE,hasoc_2020_de_486
522,522,1132433240000798720,Die Zerstörung der Grünen. https://t.co/SIYDJj...,NOT,NONE,hasoc_2020_de_3388
523,523,1127366294255357958,"RT @PParzival: ""Antideutsche"" Pseudo-linke Ide...",NOT,NONE,hasoc_2020_de_2745
524,524,1124362090460975105,Klug reden und dumm leben.,NOT,NONE,hasoc_2020_de_236


## GERMEVAL

### 2018

#### TRAIN

In [ ]:
germeval2018train = pd.read_csv('germeval2018training.txt', sep='\t', 
                               encoding='UTF-8', names=('TWEET', 'OFF', 'CAT'))
germeval2018train

,TWEET,OFF,CAT
0,"@corinnamilborn Liebe Corinna, wir würden dich...",OTHER,OTHER
1,@Martin28a Sie haben ja auch Recht. Unser Twee...,OTHER,OTHER
2,@ahrens_theo fröhlicher gruß aus der schönsten...,OTHER,OTHER
3,@dushanwegner Amis hätten alles und jeden gewä...,OTHER,OTHER
4,@spdde kein verläßlicher Verhandlungspartner. ...,OFFENSE,INSULT
...,...,...,...
5004,Gegens. Zul. zu Patenamt &amp; gegenseitige An...,OTHER,OTHER
5005,"@GlasenappHenrik Zu Merkel fällt mir nur ein, ...",OFFENSE,INSULT
5006,@KokoLores20 @krippmarie Ein richtiges Zeichen...,OFFENSE,ABUSE
5007,"@Hartes_Geld ,Honecker‘Merkel macht uns zur ,D...",OFFENSE,ABUSE


#### TEST

In [ ]:
# https://stackoverflow.com/questions/35598249/rows-are-lost-when-reading-this-tab-separated-file-with-pandas-read-csv
import csv
germeval2018test = pd.read_csv('germeval2018test.txt', sep='\t', 
                              encoding='UTF-8', quoting=csv.QUOTE_NONE, names=('TWEET', 'OFF', 'CAT'))
germeval2018test

,TWEET,OFF,CAT
0,"Meine Mutter hat mir erzählt, dass mein Vater ...",OTHER,OTHER
1,@Tom174_ @davidbest95 Meine Reaktion; |LBR| Ni...,OTHER,OTHER
2,"#Merkel rollt dem Emir von #Katar, der islamis...",OTHER,OTHER
3,„Merle ist kein junges unschuldiges Mädchen“ K...,OTHER,OTHER
4,@umweltundaktiv Asylantenflut bringt eben nur ...,OFFENSE,ABUSE
...,...,...,...
3527,"@schlabonski tja, es können einen nicht alle L...",OTHER,OTHER
3528,"@RKnillmann @lawyerberlin @AfD Aha, der Islam ...",OFFENSE,ABUSE
3529,"@podilein Mannheim, weltoffen und kunterbunt. ...",OFFENSE,ABUSE
3530,@stephanweil was ist nun mit kostenlosen Kitas...,OTHER,OTHER


In [ ]:
frames = [germeval2018train, germeval2018test]
germeval2018 = pd.concat(frames, ignore_index= True)

In [ ]:
display(germeval2018)

,TWEET,OFF,CAT
0,"@corinnamilborn Liebe Corinna, wir würden dich...",OTHER,OTHER
1,@Martin28a Sie haben ja auch Recht. Unser Twee...,OTHER,OTHER
2,@ahrens_theo fröhlicher gruß aus der schönsten...,OTHER,OTHER
3,@dushanwegner Amis hätten alles und jeden gewä...,OTHER,OTHER
4,@spdde kein verläßlicher Verhandlungspartner. ...,OFFENSE,INSULT
...,...,...,...
8536,"@schlabonski tja, es können einen nicht alle L...",OTHER,OTHER
8537,"@RKnillmann @lawyerberlin @AfD Aha, der Islam ...",OFFENSE,ABUSE
8538,"@podilein Mannheim, weltoffen und kunterbunt. ...",OFFENSE,ABUSE
8539,@stephanweil was ist nun mit kostenlosen Kitas...,OTHER,OTHER


### 2019

#### TRAIN 

In [ ]:
germeval2019train = pd.read_csv('germeval2019.training_subtask1_2_korrigiert.txt', sep='\t', 
                              encoding='UTF-8', quoting=csv.QUOTE_NONE, names=('TWEET', 'OFF', 'CAT'))
germeval2019train

,TWEET,OFF,CAT
0,@jouwatch Hat die Polizei keine Kanone mehr ? ...,OFFENSE,ABUSE
1,@de_sputnik @Saudi Arabien habt ihr mal wieder...,OFFENSE,ABUSE
2,"Glaube ich nicht , die Bundesregierung so wie ...",OFFENSE,ABUSE
3,""" Doch schockierend viele Jugendliche wissen k...",OFFENSE,PROFANITY
4,Sein Charakter war ihm wichtiger anstatt als b...,OFFENSE,PROFANITY
...,...,...,...
3990,250 Menschen auf der Demonstration gegen das D...,OTHER,OTHER
3991,Erneut Massaker an Kurdische ZivilistInnen dur...,OTHER,OTHER
3992,Hunderte Refugees haben die Grenze zur spanisc...,OTHER,OTHER
3993,Heute ab 17:00 Uhr an der Alten Oper FFM: Kund...,OTHER,OTHER


#### TEST 

In [ ]:
germeval2019test = pd.read_csv('germeval2019GoldLabelsSubtask1_2.txt', sep='\t', 
                                        encoding='UTF-8', names=['TWEET', 'OFF', 'CAT'])
germeval2019test

,TWEET,OFF,CAT
0,@JanZimmHHB @mopo Komisch das die Realitätsver...,OFFENSE,INSULT
1,@faznet @Gruene_Europa @SPDEuropa @CDU CDU ste...,OFFENSE,ABUSE
2,"@DLFNachrichten Die Gesichter, Namen, Religion...",OTHER,OTHER
3,@welt Wie verwirrt muss man sein um sich zu we...,OFFENSE,ABUSE
4,@hacker_1991 @torben_braga Weil die AfD den Fe...,OFFENSE,ABUSE
...,...,...,...
3026,Es fand aber nie eine Emanzipierungs-Phase der...,OTHER,OTHER
3027,Um es klar zu stellen: Ich will hier kein Whit...,OTHER,OTHER
3028,Und dann habe ich da noch die McArthur-Briefe ...,OTHER,OTHER
3029,al sehen wer der Ersatzmann wird. Hier könnte ...,OTHER,OTHER


#Funktionen für Preprocessing und Cleaning

Quelle:   
https://towardsai.net/p/programming/tweet-topic-modeling-part-2-cleaning-and-preprocessing-tweets  
https://gist.github.com/bicachu/09cc71bb4b0e3711eaf1556b12fa7ad7

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import pandas as pd
#import re
import gensim
from nltk.stem import WordNetLemmatizer


punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'         # define a string of punctuation symbols

# Functions to clean tweets
def remove_links(tweet):
    """Takes a string and removes web links from it"""
    tweet = re.sub(r'http\S+', '', tweet)   # remove http links
    tweet = re.sub(r'bit.ly/\\S+', '', tweet)  # remove bitly links
    tweet = tweet.strip('[link]')   # remove [links]
    tweet = re.sub(r'pic.twitter\\S+','', tweet)
    return tweet

def remove_users(tweet):
    """Takes a string and removes retweet and @user information"""
    tweet = re.sub('(RT\\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove re-tweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove tweeted at
    return tweet

def remove_hashtags(tweet):
    """Takes a string and removes any hash tags"""
    tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove hash tags
    return tweet

def remove_av(tweet):
    """Takes a string and removes AUDIO/VIDEO tags or labels"""
    tweet = re.sub('VIDEO:', '', tweet)  # remove 'VIDEO:' from start of tweet
    tweet = re.sub('AUDIO:', '', tweet)  # remove 'AUDIO:' from start of tweet
    return tweet

def tokenize(tweet):
    """Returns tokenized representation of words in lemma form excluding stopwords"""
    result = []
    for token in gensim.utils.simple_preprocess(tweet):
        if token not in stopwords and len(token) > 2:  # drops words with less than 3 characters
            result.append(lemmatize(token))
    return result

def lemmatize(token):
    """Returns lemmatization of a token"""
    return WordNetLemmatizer().lemmatize(token, pos='v')

def preprocess_tweet(tweet):
    """Main master function to clean tweets, stripping noisy characters, and tokenizing use lemmatization"""
    tweet = remove_links(tweet)
    tweet = remove_users(tweet)
    tweet = remove_hashtags(tweet)
    tweet = remove_av(tweet)
    tweet = tweet.lower()  # lower case
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\\s+', ' ', tweet)  # remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    tweet_token_list = tokenize(tweet)  # apply lemmatization and tokenization
    tweet = ' '.join(tweet_token_list)
    return tweet

def basic_clean(tweet):
    """Main master function to clean tweets only without tokenization or removal of stopwords"""
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = remove_hashtags(tweet)
    tweet = remove_av(tweet)
    tweet = tweet.lower()  # lower case
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\\s+', ' ', tweet)  # remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    tweet = re.sub('📝 …', '', tweet)
    return tweet

def tokenize_tweets(df):
    """Main function to read in and return cleaned and preprocessed dataframe.
    This can be used in Jupyter notebooks by importing this module and calling the tokenize_tweets() function
    Args:
        df = data frame object to apply cleaning to
    Returns:
        pandas data frame with cleaned tokens
    """

    df['tokens'] = df.tweet.apply(preprocess_tweet)
    num_tweets = len(df)
    print('Complete. Number of Tweets that have been cleaned and tokenized : {}'.format(num_tweets))
    return df

-----------------

# GermEval2018

### Model (mit Vorverarbeitung und wenigen hinzugefügten Stoppwörtern)

In [ ]:
ge18train = germeval2018train.copy()


In [ ]:
ge18train

,TWEET,OFF,CAT
0,"@corinnamilborn Liebe Corinna, wir würden dich...",OTHER,OTHER
1,@Martin28a Sie haben ja auch Recht. Unser Twee...,OTHER,OTHER
2,@ahrens_theo fröhlicher gruß aus der schönsten...,OTHER,OTHER
3,@dushanwegner Amis hätten alles und jeden gewä...,OTHER,OTHER
4,@spdde kein verläßlicher Verhandlungspartner. ...,OFFENSE,INSULT
...,...,...,...
5004,Gegens. Zul. zu Patenamt &amp; gegenseitige An...,OTHER,OTHER
5005,"@GlasenappHenrik Zu Merkel fällt mir nur ein, ...",OFFENSE,INSULT
5006,@KokoLores20 @krippmarie Ein richtiges Zeichen...,OFFENSE,ABUSE
5007,"@Hartes_Geld ,Honecker‘Merkel macht uns zur ,D...",OFFENSE,ABUSE


In [ ]:
ge18train = ge18train.rename(columns={'TWEET':'tweet'})

In [ ]:
ge18train_tokenized = tokenize_tweets(ge18train)
ge18train_tokenized

Complete. Number of Tweets that have been cleaned and tokenized : 5009


,tweet,OFF,CAT,tokens
0,"@corinnamilborn Liebe Corinna, wir würden dich...",OTHER,OTHER,liebe corinna gerne moderatorin gewinnen wärst...
1,@Martin28a Sie haben ja auch Recht. Unser Twee...,OTHER,OTHER,recht tweet bverfg sachleistungen ausschließt ...
2,@ahrens_theo fröhlicher gruß aus der schönsten...,OTHER,OTHER,fröhlicher gruß schönsten stadt welt theo
3,@dushanwegner Amis hätten alles und jeden gewä...,OTHER,OTHER,amis hätten gewählt hillary wollten fortsetzun...
4,@spdde kein verläßlicher Verhandlungspartner. ...,OFFENSE,INSULT,verläßlicher nachkarteln schickt stümper verse...
...,...,...,...,...
5004,Gegens. Zul. zu Patenamt &amp; gegenseitige An...,OTHER,OTHER,gegens zul patenamt gegenseitige anerk firmung...
5005,"@GlasenappHenrik Zu Merkel fällt mir nur ein, ...",OFFENSE,INSULT,merkel fällt verräterin immer verräterin selbs...
5006,@KokoLores20 @krippmarie Ein richtiges Zeichen...,OFFENSE,ABUSE,richtiges zeichen unserer nachbarn schleichend...
5007,"@Hartes_Geld ,Honecker‘Merkel macht uns zur ,D...",OFFENSE,ABUSE,honecker merkel macht ddr klagen angenommen ze...


In [ ]:
ge18_docs = ge18train_tokenized['tokens']


In [ ]:
ge18_docs

0       liebe corinna gerne moderatorin gewinnen wärst...
1       recht tweet bverfg sachleistungen ausschließt ...
2               fröhlicher gruß schönsten stadt welt theo
3       amis hätten gewählt hillary wollten fortsetzun...
4       verläßlicher nachkarteln schickt stümper verse...
                              ...                        
5004    gegens zul patenamt gegenseitige anerk firmung...
5005    merkel fällt verräterin immer verräterin selbs...
5006    richtiges zeichen unserer nachbarn schleichend...
5007    honecker merkel macht ddr klagen angenommen ze...
5008    warum wurden chaoten sofort frischer tat verha...
Name: tokens, Length: 5009, dtype: object

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer



In [ ]:
pip install --upgrade tbb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
vectorizer_model = CountVectorizer(stop_words=stopwords)


In [ ]:
topic_model = BERTopic(language="german", vectorizer_model=vectorizer_model, calculate_probabilities=False, nr_topics="auto")


In [ ]:
topics, probs = topic_model.fit_transform(ge18_docs)

In [ ]:
ge18_train_model_topic_df = topic_model.get_topic_info()
ge18_train_model_topic_df

,Topic,Count,Name
0,-1,2033,-1_deutschland_immer_merkel_deutschen
1,0,1210,0_merkel_deutschland_deutschen_deutsche
2,1,126,1_flüchtlinge_asylanten_asyl_migranten
3,2,125,2_islam_moslems_muslimischen_moslem
4,3,92,3_politiker_politik_politikern_demokratie
5,4,85,4_medien_journalismus_propaganda_news
6,5,82,5_grünen_grüne_grün_rot
7,6,82,6_türkei_türken_türke_türkischen
8,7,80,7_tweet_twitter_facebook_retweeten
9,8,71,8_geld_zahlen_millionen_shop


In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart(top_n_topics=100)

In [ ]:
#topic_model.save("tm_germeval2018_processed")

### _ auf Test-GermEval2018

In [ ]:
ge18test = germeval2018test.copy()
ge18test = ge18test.rename(columns={'TWEET':'tweet'})


In [ ]:
#test_docs = ge18test['tweet'].copy()


In [ ]:
ge18test_tokenized = tokenize_tweets(ge18test)


Complete. Number of Tweets that have been cleaned and tokenized : 3532


In [ ]:
test_docs = ge18test_tokenized['tokens']

In [ ]:
predictions = topic_model.transform(test_docs)
predictions

([-1,
  2,
  -1,
  0,
  12,
  -1,
  -1,
  -1,
  -1,
  -1,
  0,
  -1,
  0,
  -1,
  22,
  13,
  27,
  -1,
  -1,
  0,
  -1,
  11,
  -1,
  -1,
  0,
  1,
  0,
  0,
  -1,
  0,
  0,
  0,
  0,
  14,
  -1,
  -1,
  -1,
  3,
  33,
  8,
  0,
  30,
  -1,
  0,
  -1,
  -1,
  0,
  -1,
  0,
  40,
  2,
  7,
  -1,
  0,
  13,
  -1,
  -1,
  -1,
  0,
  1,
  0,
  -1,
  -1,
  -1,
  -1,
  -1,
  0,
  -1,
  -1,
  -1,
  0,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  5,
  -1,
  0,
  10,
  -1,
  -1,
  -1,
  -1,
  40,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  5,
  -1,
  29,
  -1,
  17,
  -1,
  -1,
  46,
  27,
  1,
  -1,
  0,
  0,
  -1,
  39,
  0,
  0,
  16,
  -1,
  0,
  0,
  0,
  -1,
  0,
  -1,
  -1,
  0,
  0,
  24,
  0,
  -1,
  33,
  0,
  0,
  -1,
  2,
  -1,
  13,
  -1,
  -1,
  -1,
  -1,
  0,
  0,
  24,
  5,
  0,
  0,
  -1,
  -1,
  -1,
  10,
  -1,
  7,
  -1,
  6,
  -1,
  -1,
  -1,
  -1,
  -1,
  2,
  -1,
  1,
  -1,
  -1,
  -1,
  -1,
  0,
  0,
  -1,
  -1,
  -1,
  5,
  5,
  -1,
  0,
  -1,
  0,
  19,
  45,
  -1,
  0,

In [ ]:
df_predictions = pd.DataFrame(predictions[0])

In [ ]:
df_predictions

,0
0,-1
1,2
2,-1
3,0
4,12
...,...
3527,0
3528,-1
3529,-1
3530,-1


In [ ]:
#ge18test_predictions

In [ ]:
ge18test['pred'] = pd.Series(predictions[0])


In [ ]:
ge18test

,tweet,OFF,CAT,tokens,pred
0,"Meine Mutter hat mir erzählt, dass mein Vater ...",OTHER,OTHER,mutter erzählt vater gewählt homo ehe,-1
1,@Tom174_ @davidbest95 Meine Reaktion; |LBR| Ni...,OTHER,OTHER,reaktion moslem terrorist moslem glaubt überli...,2
2,"#Merkel rollt dem Emir von #Katar, der islamis...",OTHER,OTHER,rollt emir islamistischen terror unterstützt r...,-1
3,„Merle ist kein junges unschuldiges Mädchen“ K...,OTHER,OTHER,merle junges unschuldiges mädchen kch,0
4,@umweltundaktiv Asylantenflut bringt eben nur ...,OFFENSE,ABUSE,asylantenflut bringt eben negative deutschland...,12
...,...,...,...,...,...
3527,"@schlabonski tja, es können einen nicht alle L...",OTHER,OTHER,tja lieben immer geben mag,0
3528,"@RKnillmann @lawyerberlin @AfD Aha, der Islam ...",OFFENSE,ABUSE,aha islam religion,-1
3529,"@podilein Mannheim, weltoffen und kunterbunt. ...",OFFENSE,ABUSE,mannheim weltoffen kunterbunt strunzdumme verb...,-1
3530,@stephanweil was ist nun mit kostenlosen Kitas...,OTHER,OTHER,kostenlosen kitas groko,-1


In [ ]:
topic_count_train_ge18_test_ge18 = pd.DataFrame(ge18test['pred'].value_counts())
topic_count_train_ge18_test_ge18

,pred
-1,2029
0,697
1,90
2,84
5,55
7,47
8,39
27,31
13,27
3,27


### _ auf Test-GermEval2019

In [ ]:
ge19test = germeval2019test.copy()
ge19test = ge19test.rename(columns={'TWEET':'tweet'})

In [ ]:
ge19test_tokenized = tokenize_tweets(ge19test)


Complete. Number of Tweets that have been cleaned and tokenized : 3031


In [ ]:
test_docs = ge19test_tokenized['tokens']

In [ ]:
predictions = topic_model.transform(test_docs)


In [ ]:
df_predictions = pd.DataFrame(predictions[0])

In [ ]:
df_predictions

,0
0,-1
1,-1
2,0
3,-1
4,-1
...,...
3026,6
3027,-1
3028,-1
3029,-1


In [ ]:
ge19test['pred'] = pd.Series(predictions[0])


In [ ]:
ge19test

,tweet,OFF,CAT,tokens,pred
0,@JanZimmHHB @mopo Komisch das die Realitätsver...,OFFENSE,INSULT,komisch immer linken erbärmlichen correctiv ac...,-1
1,@faznet @Gruene_Europa @SPDEuropa @CDU CDU ste...,OFFENSE,ABUSE,cdu steht seid strauss kohl schäuble vdleyen k...,-1
2,"@DLFNachrichten Die Gesichter, Namen, Religion...",OTHER,OTHER,gesichter namen religion täter aufklärung beit...,0
3,@welt Wie verwirrt muss man sein um sich zu we...,OFFENSE,ABUSE,verwirrt weigern tatsachen anzuerkennen hass f...,-1
4,@hacker_1991 @torben_braga Weil die AfD den Fe...,OFFENSE,ABUSE,afd ferkelfunk abschaffen deshalb linken zecke...,-1
...,...,...,...,...,...
3026,Es fand aber nie eine Emanzipierungs-Phase der...,OTHER,OTHER,fand nie emanzipierungs phase zivil bevölkerun...,6
3027,Um es klar zu stellen: Ich will hier kein Whit...,OTHER,OTHER,klar stellen white wash betreiben hätte unters...,-1
3028,Und dann habe ich da noch die McArthur-Briefe ...,OTHER,OTHER,mcarthur briefe sinn eindruck damals moderner ...,-1
3029,al sehen wer der Ersatzmann wird. Hier könnte ...,OTHER,OTHER,sehen wer ersatzmann ggf aktuelle richtung kor...,-1


In [ ]:
ge19test['pred'].value_counts()

-1     1823
 0      450
 7       81
 13      52
 8       48
 1       45
 31      42
 14      33
 10      31
 4       30
 2       29
 27      26
 3       25
 24      24
 16      20
 26      18
 20      18
 39      17
 5       17
 44      17
 41      14
 15      13
 6       12
 29      12
 11      11
 25      10
 35      10
 12       9
 23       9
 19       8
 21       8
 33       7
 32       7
 34       7
 22       7
 9        7
 18       5
 46       4
 42       4
 38       4
 36       4
 30       3
 40       3
 45       2
 28       2
 17       2
 37       1
Name: pred, dtype: int64

In [ ]:
topic_count_train_ge18_test_ge19 = pd.DataFrame(ge19test['pred'].value_counts())
topic_count_train_ge18_test_ge19

,pred
-1,1823
0,450
7,81
13,52
8,48
1,45
31,42
14,33
10,31
4,30


### _ auf Test-Hasoc2019

In [ ]:
h19test = hasoc19test.copy()
h19test = h19test.rename(columns={'text':'tweet'})

In [ ]:
h19test_tokenized = tokenize_tweets(h19test)


Complete. Number of Tweets that have been cleaned and tokenized : 850


In [ ]:
test_docs = h19test_tokenized['tokens']

In [ ]:
predictions = topic_model.transform(test_docs)


In [ ]:
df_predictions = pd.DataFrame(predictions[0])

In [ ]:
df_predictions

,0
0,-1
1,-1
2,0
3,-1
4,-1
...,...
845,-1
846,24
847,-1
848,-1


In [ ]:
h19test['pred'] = pd.Series(predictions[0])


In [ ]:
#h19test['pred'] = pd.Series(predictions[0])


In [ ]:
h19test

,text_id,tweet,task_1,task_2,tokens,pred
0,hasoc_de_5270,"Die Grüne Intelligenz Bestie 'Eckhardt', Prang...",HOF,HATE,grüne intelligenz bestie eckhardt prangert tic...,-1
1,hasoc_de_5271,"Das Deutschlandlied, Sinn und Unsinn https:/...",NOT,NONE,deutschlandlied sinn unsinn,-1
2,hasoc_de_5272,#Eurovision https://t.co/24T8fzUI44,NOT,NONE,,0
3,hasoc_de_5273,Unsere ESC Kandidatinnen haben selbst Schuld a...,HOF,OFFN,esc kandidatinnen schuld point public wer jung...,-1
4,hasoc_de_5274,Scheinbar bekommt Sarah's Conner's Sohn Vincen...,HOF,OFFN,scheinbar bekommt sarah conner sohn vincent mä...,-1
...,...,...,...,...,...,...
845,hasoc_de_6115,#Erdogan ruft seine Mops wegen Untauglichkeit ...,HOF,OFFN,ruft mop wegen untauglichkeit zurück wurde zur...,-1
846,hasoc_de_6116,Ich würde der Linken Chefin Kipping die 1.000 ...,HOF,OFFN,linken chefin kip euro geben wären bedeutend b...,24
847,hasoc_de_6117,Ausländer sind Top &amp; Trumpf Schon aufgefa...,HOF,OFFN,ausländer top trumpf aufgefallen ausländer sag...,-1
848,hasoc_de_6118,Wie der Regierungsjet die Regierungsflieger wi...,HOF,OFFN,regierungsjet wiederspiegelt probleme steuerun...,-1


In [ ]:
h19test['pred'].value_counts()

-1     443
 0     173
 1      40
 2      16
 8      15
 7      13
 5      11
 23     11
 13     11
 3      11
 12      9
 46      8
 4       8
 31      6
 20      5
 44      5
 24      5
 27      5
 15      5
 18      5
 22      5
 26      4
 6       4
 11      4
 21      3
 35      3
 16      3
 30      3
 25      3
 39      2
 19      2
 10      2
 43      2
 14      1
 17      1
 40      1
 34      1
 45      1
Name: pred, dtype: int64

In [ ]:
topic_count_train_ge18_test_h19 = pd.DataFrame(h19test['pred'].value_counts())
topic_count_train_ge18_test_h19

,pred
-1,443
0,173
1,40
2,16
8,15
7,13
5,11
23,11
13,11
3,11


### _ auf Test-Hasoc2020

In [ ]:
h20test = hasoc20test.copy()
h20test = h20test.rename(columns={'text':'tweet'})

In [ ]:
h20test_tokenized = tokenize_tweets(h20test)


Complete. Number of Tweets that have been cleaned and tokenized : 526


In [ ]:
test_docs = h20test_tokenized['tokens']

In [ ]:
predictions = topic_model.transform(test_docs)


In [ ]:
df_predictions = pd.DataFrame(predictions[0])

In [ ]:
df_predictions

,0
0,-1
1,0
2,33
3,-1
4,-1
...,...
521,-1
522,5
523,-1
524,10


In [ ]:
h20test['pred'] = pd.Series(predictions[0])


In [ ]:
predictions = topic_model.transform(test_docs)


In [ ]:
h20test['pred'] = pd.Series(predictions[0])


In [ ]:
h20test

,Unnamed: 0,tweet_id,tweet,task1,task2,ID,tokens,pred
0,0,1129095874242650112,@derCarsti Boykottieren hört sich besser an. 💙💙,NOT,NONE,hasoc_2020_de_1053,boykottieren hört besser,-1
1,1,1129004308396236800,"RT @ibikus31: Es wird spekuliert, ob Merkel ei...",NOT,NONE,hasoc_2020_de_671,spekuliert merkel amt brüssel übernimmt wäre s...,0
2,2,1130896929355907080,Hat #Hitler wirklich den Krieg in der Wüste ve...,NOT,NONE,hasoc_2020_de_2977,wirklich krieg wüste verloren schlagzeile reag...,33
3,3,1132251534329307136,RT @Beatrix_vStorch: #May tritt in UK unter Tr...,NOT,NONE,hasoc_2020_de_1746,tritt tränen zurück zurücktritt tränen freuden...,0
4,4,1124941869115498496,@justmeDoro Eher nicht. Das Gänse hauen wieder...,NOT,NONE,hasoc_2020_de_2416,eher gänse hauen warmen,-1
...,...,...,...,...,...,...,...,...
521,521,1124809878546128897,RT @ChanMachtSo: SCHMERZEN!!!! Au!!! Mein Gehi...,NOT,NONE,hasoc_2020_de_486,schmerzen gehirn auuuuuu,-1
522,522,1132433240000798720,Die Zerstörung der Grünen. https://t.co/SIYDJj...,NOT,NONE,hasoc_2020_de_3388,zerstörung grünen,5
523,523,1127366294255357958,"RT @PParzival: ""Antideutsche"" Pseudo-linke Ide...",NOT,NONE,hasoc_2020_de_2745,antideutsche pseudo linke ideologie sache syst...,-1
524,524,1124362090460975105,Klug reden und dumm leben.,NOT,NONE,hasoc_2020_de_236,klug reden dumm leben,10


In [ ]:
h20test['pred'].value_counts()

-1     284
 0     100
 5      30
 10     27
 13     18
 2       8
 24      8
 8       8
 1       4
 3       4
 16      4
 25      3
 28      3
 39      2
 11      2
 4       2
 9       2
 6       2
 27      2
 7       1
 33      1
 46      1
 12      1
 45      1
 42      1
 26      1
 36      1
 32      1
 30      1
 23      1
 44      1
 31      1
Name: pred, dtype: int64

In [ ]:
topic_count_train_ge18_test_h20 = pd.DataFrame(h20test['pred'].value_counts())
topic_count_train_ge18_test_h20

,pred
-1,284
0,100
5,30
10,27
13,18
2,8
24,8
8,8
1,4
3,4
